In [ ]:
#install.packages("DBI")
library(DBI)
#install.packages("httr")
#library(httr)

# load OMOP dbutils.R
source('../R/dbutils.R')

In [ ]:
# check the connection
con <- db_con()
dbListTables(con)

In [157]:
# retrieve EPA monitor sites through file download
fetch_EPA_sites_from_file <- function() {
    
     # all sites (i.e. monitor locations)
    base_download_url <- "https://aqs.epa.gov/aqsweb/airdata/"
    file_download <- "aqs_sites.zip"
    file_name <- "aqs_sites.csv"

    temp <- tempfile()
    download.file(paste(base_download_url,file_download,sep=""),temp)
    data <- read.csv(unz(temp, file_name), header=TRUE)
    unlink(temp)
    
    return (data.frame(data))
    
}

# load sites into database
dbload_EPA_sites <- function(sites,geom_tablename) {
    
    loaded <- list()
    done <- by(EPA_sites, seq_len(nrow(EPA_sites)), function(site) {

        if (! site$Site.Number %in% loaded) {

            # create location with geometry
            loaded <- c(site$Site.Number,loaded)
            sql_createsite <- paste(
                'INSERT INTO',geom_tablename,' ("geom_record_id","name","geom_wgs84") 
                VALUES (DEFAULT,$3,ST_SetSRID(ST_MakePoint($2,$1),4326))
                RETURNING geom_record_id'
            )
            res <- dbSendQuery(con,sql_createsite)
            dbBind(res, list(site$Latitude,site$Longitude,paste(site$Site.Number,":",site$Local.Site.Name)))
            site_ID <- dbFetch(res)[1,1]
            dbClearResult(res)

        }

    })
    
}

In [164]:
# load sites
EPA_sites = fetch_EPA_sites_from_file()
dbload(EPA_sites,'geo_us_epa_aqs_sites')

In [ ]:
# retrieve EPA AQS data through their API
# TODO: create institutional email and credentials for API key
# NOTE: this could also be done through file download: 
#.      file_download <- "annual_conc_by_monitor_2020.zip"
fetch_EPA_AQS_from_api <- function(options) {

    base_api_url <- "https://aqs.epa.gov/data/api/annualData/"

    request_url <- paste0(
        base_api_url,options[['api_query_type']],"?", 
        "email=",options[['api_email']], 
        "&key=",options[['api_key']],
        "&param=",options[['api_param']],
        "&bdate=",options[['api_bdate']],
        "&edate=",options[['api_bdate']],
        "&state=",options[['api_state']]
    )
    
    res <- jsonlite::fromJSON(request_url, simplifyDataFrame = TRUE)
    # TODO: error checking on response
    data <- res$Data
    return(data)
    
}

In [ ]:
options <- list(
    api_query_type = "byState",
    api_email = "test@aqs.api",
    api_key = "test",
    api_param = "88101,88502",  # PM2.5 88101,88502; NO2 42602
                                # https://aqs.epa.gov/aqsweb/documents/codetables/methods_all.html
    api_bdate = "20180115",     # begin date
    api_edate = "20180115",     # end date: set to same as bdate for whole year
    api_state = "12"            # Florida
)
aqs_data = fetch_EPA_AQS_from_api(options)

In [ ]:
aqs_data

In [130]:
EPA_monitor_sites

State.Code,County.Code,Site.Number,Latitude,Longitude,Datum,Elevation,Land.Use,Location.Setting,Site.Established.Date,⋯,Owning.Agency,Local.Site.Name,Address,Zip.Code,State.Name,County.Name,City.Name,CBSA.Name,Tribe.Name,Extraction.Date
<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
01,1,1,32.43746,-86.47289,WGS84,64.00,RESIDENTIAL,SUBURBAN,1974-05-01,⋯,Al Dept Of Env Mgt,,"KING ARTHUR TRAILER COURT, PRATTVILLE,AL",36067,Alabama,Autauga,Prattville,"Montgomery, AL",,2020-11-24
01,1,2,32.42847,-86.44358,WGS84,0.00,AGRICULTURAL,RURAL,1980-01-01,⋯,Al Dept Of Env Mgt,,COUNTY RD 4 PRATTVILLE EXPERIMENT ST,NA,Alabama,Autauga,Prattville,"Montgomery, AL",,2020-11-24
01,1,3,32.33266,-86.79152,WGS84,41.00,FOREST,RURAL,1989-08-31,⋯,Al Dept Of Env Mgt,,"1170 COUNTY RD.15 SO., SELMA, AL. 36701",36003,Alabama,Autauga,Not in a City,"Montgomery, AL",,2020-11-24
01,3,1,0.00000,0.00000,NAD27,0.00,UNKNOWN,RURAL,1959-01-01,⋯,US EPA National Exposure Research Lab,,CHANDLER RESIDENCE FT MORGAN STATE PARK,NA,Alabama,Baldwin,Not in a city,"Daphne-Fairhope-Foley, AL",,2020-11-24
01,3,2,30.55237,-87.70691,WGS84,0.00,COMMERCIAL,RURAL,1974-01-01,⋯,Al Dept Of Env Mgt,,COR PENNSYLVANIA & CHICAGO (ROBERTSDALE),36567,Alabama,Baldwin,Robertsdale,"Daphne-Fairhope-Foley, AL",,2020-11-24
01,3,3,30.55547,-87.71360,WGS84,49.00,COMMERCIAL,URBAN AND CENTER CITY,1977-08-12,⋯,Al Dept Of Env Mgt,,"RACINE AND MICHIGAN ST, ROBERTSDALE, AL.",NA,Alabama,Baldwin,Not in a city,"Daphne-Fairhope-Foley, AL",,2020-11-24
01,3,10,30.49748,-87.88026,NAD83,37.19,COMMERCIAL,SUBURBAN,2000-01-01,⋯,Al Dept Of Env Mgt,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE, ALABAMA",36532,Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",,2020-11-24
01,5,1,31.89127,-85.13521,WGS84,85.00,COMMERCIAL,SUBURBAN,1974-01-01,⋯,Al Dept Of Env Mgt,,601 EAST BARBOUR US HGWY 82,NA,Alabama,Barbour,Eufaula,"Eufaula, AL-GA",,2020-11-24
01,5,2,31.66430,-85.60617,WGS84,143.26,FOREST,SUBURBAN,2003-10-06,⋯,Al Dept Of Env Mgt,CLIO,"437 COUNTY ROAD 8, CLIO, ALABAMA",NA,Alabama,Barbour,Not in a city,"Eufaula, AL-GA",,2020-11-24
